# Ume Multi-Modal Embeddings Tutorial

This notebook show how to use the Universal Molecular Encoder (Ume) to generate embeddings for different molecular modalities: amino acids, SMILES, and nucleotides. Stay tuned for 3D coordinate embeddings and more!

In [1]:
from lobster.model import Ume

ume = Ume()

print(f"Supported modalities: {ume.modalities}")
print(f"Vocab size: {len(ume.get_vocab())}")

/Users/zadorozk/Desktop/code/lobster/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Supported modalities: ['SMILES', 'amino_acid', 'nucleotide', '3d_coordinates']
Vocab size: 1536


### Load from checkpoint

In [ ]:
checkpoint = "ume-checkpoints/last.ckpt" # Replace with the correct checkpoint path

ume = Ume.load_from_checkpoint(checkpoint)

### 1. Protein sequences

Embed sample protein sequence to get full sequence embedding or per-residue embeddings.

In [3]:
# Example protein sequences
protein_sequences = [
    "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",  # Sample protein fragment
    "MLSRAVCGTSRQLAPVLAYLGSRQKHSLPDLPYDYGALEPHINAQIMQLHHSKHHAAYVNNLNVTEEKYQEALAKGDVTAQIALQPALKFNGGGHINHSIFWTNLSPNGGGEPKGELLEAIKRDFGSFDKFKEKLTAASVGVQGSGWGWLGFNKERGHLQIAACPNQDPLQGTTGLIPLLGIDVWEHAYYLQYKNVRPDYLKAIWNVINWENVTERYMACKK",  # Human SOD1
    "MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKVKAHGKKVLGAFSDGLAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH"  # Hemoglobin beta chain
]

# Get embeddings for protein sequences
protein_embeddings = ume.get_embeddings(protein_sequences, modality="amino_acid")
print(f"Protein embeddings shape: {protein_embeddings.shape}")

# Get token-level embeddings (without aggregation)
protein_residue_embeddings = ume.get_embeddings(protein_sequences, modality="amino_acid", aggregate=False)
print(f"Protein token-level embeddings shape: {protein_residue_embeddings.shape}")

Protein embeddings shape: torch.Size([3, 384])
Protein token-level embeddings shape: torch.Size([3, 512, 384])


### 2. SMILES
SMILES strings are a text-based representation of molecular structures. Here we embed common drug molecules.


In [4]:
# Example SMILES strings for common molecules
smiles_examples = [
    "CC(=O)OC1=CC=CC=C1C(=O)O",  # Aspirin
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # Caffeine
    "COC1=CC=C(CCN)C=C1",  # Dopamine
    "C1=CC=C(C(=C1)C(=O)O)O",  # Salicylic acid
    "CCC(CC)COC(=O)[C@H](C)N[P@](=O)(OC[C@H]1O[C@@H]([C@H]([C@@H]1O)O)N1C=NC2=C1N=CN=C2N)OC1=CC=CC=C1"  # Remdesivir
]

# Get embeddings for SMILES
smiles_embeddings = ume.get_embeddings(smiles_examples, modality="SMILES")
print(f"SMILES embeddings shape: {smiles_embeddings.shape}")

SMILES embeddings shape: torch.Size([5, 384])


### 3. Nucleotides

Embed example DNA/RNA sequences.

In [6]:
# Example DNA/RNA sequences
nucleotide_sequences = [
    "ATGCATGCATGCATGCATGCATGCATGCATGCATGCATGCATGCATGC",  
    "GATTACACAGTGCTTGACCCGATCGATCGATCGATCGATCGATCGATCGA",  
    "AUGCUAUGCUAGCUAGCUAGCUAGCUAUGCUAGCUAUGCUAGCUAUC"  # RNA sequence 
]

# Get embeddings for nucleotide sequences
nucleotide_embeddings = ume.get_embeddings(nucleotide_sequences, modality="nucleotide")
print(f"Nucleotide embeddings shape: {nucleotide_embeddings.shape}")

Nucleotide embeddings shape: torch.Size([3, 384])


## Using Embeddings for Downstream Tasks
Quick example of using molecular embeddings for a classification task.

In [ ]:
# Dummy classification setup 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# SMILES with some property labels
inputs = ["CC(=O)OC1=CC=CC=C1C(=O)O", "CN1C=NC2=C1C(=O)N(C(=O)N2C)C", "COC1=CC=C(CCN)C=C1", 
              "C1=CC=C(C(=C1)C(=O)O)O", "CC12CCC(CC1)CC(C2)C(C)CN"]
labels = [0, 1, 0, 1, 0]  # Binary classification example

# Get embeddings
X = ume.get_embeddings(inputs, modality="SMILES").cpu().numpy()
y = np.array(labels)

# Train a simple classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0



/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(